In [ ]:
#ライブラリ
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
# CSV読み込み・データ整理
df = pd.read_csv('/content/drive/MyDrive/spam.csv', encoding='latin-1')
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)
df.rename(columns={"v1":"label", "v2":"text"}, inplace=True)
df.head()
X = pd.DataFrame(df["text"])
y = pd.DataFrame(df["label"])
#7：3でデータセットを分割
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=42)
# 単語の出現回数取得
vectorizer = CountVectorizer(min_df=5)
# vectorizerをもとにエンコード
X_train_vec = vectorizer.fit_transform(X_train["text"])
X_test_vec = vectorizer.transform(X_test["text"])
y_train = y_train["label"]
y_test = y_test["label"]

In [ ]:
class NaiveBayesClassifier:
    def __init__(self):
        self.classes= ["ham","spam"] #各カテゴリ
        self.length = 1416 #全学習データの中に出現する、重複を除いた全語句の数(語彙)→N_all
        self.prob_priors = [0.0, 0.0] #全学習データのうち、ham/spamに属するデータの割合→r_cat
        self.word_likelihoods = np.zeros((2, self.length)) #各カテゴリにおいて、ある語句が起こる確率→P(cat)

    def fit(self,X,y):
        for i, c in enumerate(self.classes):
            self.prob_priors[i] = np.mean(y == c)
            class_index = [k for k, j in enumerate(y) if j == c]
            X_c = X[class_index]
            N_w = np.squeeze(np.array(np.sum(X_c, axis=0)))#各カテゴリに属する、ある語句の数→N_w
            N_cat = np.sum(N_w)#各カテゴリに属する、全ての語句の数→N_cat
            for idx, k in enumerate(N_w):
                self.word_likelihoods[i][idx] = (k + 1)/(N_cat + self.length)#数式の右項の第二項

    def predict(self, X):
        prediction = [] #最終的な予測カテゴリを格納する
        classes = [0,1]
        p_cat=[0.0, 0.0] #各カテゴリの尤度
        X = (X > 0).astype(int).toarray()#Xを語彙リストに基づいて0/1のリストに変換
        for x in X : #xにはバイナリデータが入っており、各単語が出現したか否かがわかる
            for c in classes:
                p_cat[c] = np.log(self.prob_priors[c])
                for idx, i in enumerate(x):
                    if i == 1:
                        p_cat[c]+= np.log(self.word_likelihoods[c][idx])
            #カテゴリ分類
            if p_cat[0] > p_cat[1]:
                prediction.append("ham")
            else :
                prediction.append("spam")
        return prediction



In [ ]:
#精度検証
def acc(pred, ans):
    correct = 0
    for idx, k in enumerate(ans):
        if k == pred[idx]:
            correct = correct + 1
    print(correct / len(pred)) #予測カテゴリが真のカテゴリと一致した確率
#実装
nb = NaiveBayesClassifier()
#学習
n = nb.fit(X_train_vec, y_train)
#予測
y_pred = nb.predict(X_test_vec)
#精度
acc(y_pred, y_test) #0.9802631578947368
print(y_pred) #確認用
print(y_test.tolist()) #確認用